# Data Ingestion Parquet To PostgreSQL

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('CO2 Emission Country.parquet')

In [3]:
df.head()

,Location,% of global total,Fossil emissions 2023,Fossil emissions 2000,% change from 2000
0,China,34.0%,"13,259.64","3,666.95",+262%
1,United States,12.0%,"4,682.04","5,928.97",−21%
2,India,7.6%,"2,955.18",995.65,+197%
3,European Union,6.4%,"2,512.07","3,563.26",−30%
4,Russia,5.3%,"2,069.50","1,681.14",+23%


In [4]:
df.describe()

,Location,% of global total,Fossil emissions 2023,Fossil emissions 2000,% change from 2000
count,211,211,211,211,211
unique,211,47,201,194,156
top,China,0.02%,0.10,0.06,−33%
freq,1,24,3,4,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Location               211 non-null    object
 1   % of global total      211 non-null    object
 2   Fossil emissions 2023  211 non-null    object
 3   Fossil emissions 2000  211 non-null    object
 4   % change from 2000     211 non-null    object
dtypes: object(5)
memory usage: 8.4+ KB


# Remove % string from [% of global total]

In [6]:
df['% of global total'] = df['% of global total'].str.replace('%', '', regex=False)

In [7]:
df['% of global total'].head()

0    34.0
1    12.0
2     7.6
3     6.4
4     5.3
Name: % of global total, dtype: object

# Convert object to numeric [% of global total]

In [8]:
df['% of global total'] = pd.to_numeric(df['% of global total'], errors='raise')

In [9]:
df['% of global total'].head()

0    34.0
1    12.0
2     7.6
3     6.4
4     5.3
Name: % of global total, dtype: float64

# Remove string ',' from [Fossil Emissin 2023]

In [10]:
df['Fossil emissions 2023'] = df['Fossil emissions 2023'].str.replace(',', '', regex=False)

# Conver object to numeric [Fossil emission 2023]

In [11]:
df['Fossil emissions 2023'] = pd.to_numeric(df['Fossil emissions 2023'], errors='raise')

In [13]:
df['Fossil emissions 2023'].head()

0    13259.64
1     4682.04
2     2955.18
3     2512.07
4     2069.50
Name: Fossil emissions 2023, dtype: float64

# Remove ',' & 'no' string from [Fossil emissions 2000]

In [16]:
df['Fossil emissions 2000'] = df['Fossil emissions 2000'].str.replace(',', '', regex=False)

In [17]:
df['Fossil emissions 2000'] = df['Fossil emissions 2000'].str.replace('no', '', regex=False)

# Convert to numeric [Fossil emisions 2000]

In [20]:
df['Fossil emissions 2000'] = pd.to_numeric(df['Fossil emissions 2000'], errors='raise')

In [21]:
df['Fossil emissions 2000'].head()

0    3666.95
1    5928.97
2     995.65
3    3563.26
4    1681.14
Name: Fossil emissions 2000, dtype: float64

# Remove '%' '+' string from [% change from 2000]

In [22]:
df['% change from 2000'] = df['% change from 2000'].str.replace('%', '', regex=False)

In [23]:
df['% change from 2000'] = df['% change from 2000'].str.replace('+', '', regex=False)

In [24]:
df['% change from 2000'] = df['% change from 2000'].str.replace("−", "-")

# Remove ',' & 'change' from [% change from 2000]

In [25]:
df['% change from 2000'] = df['% change from 2000'].str.replace(',', '', regex=False)

In [26]:
df['% change from 2000'] = df['% change from 2000'].str.replace('change', '', regex=False)

# Convert objrct to numeric [% change from 2000]

In [27]:
df['% change from 2000'] = pd.to_numeric(df['% change from 2000'], errors='raise')

In [28]:
df['% change from 2000'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 211 entries, 0 to 210
Series name: % change from 2000
Non-Null Count  Dtype  
--------------  -----  
210 non-null    float64
dtypes: float64(1)
memory usage: 1.8 KB


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Location               211 non-null    object 
 1   % of global total      211 non-null    float64
 2   Fossil emissions 2023  211 non-null    float64
 3   Fossil emissions 2000  210 non-null    float64
 4   % change from 2000     210 non-null    float64
dtypes: float64(4), object(1)
memory usage: 8.4+ KB


# Import python library to connect & interaction with postgreSQL

In [30]:
from sqlalchemy import create_engine
import psycopg2

# Create database connection

In [31]:
db_config = {
    'host':'localhost',
    'database':'parquetpostgres',
    'user':'postgres',
    'password':'postgres'
}

# Create table name

In [32]:
table_name = 'co2_emission'

# Load data frame into postgreSQL

In [33]:
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        location VARCHAR(100),
        global_total DECIMAL(10,2),
        emission_2023 DECIMAL(10,2),
        emission_2000 DECIMAL(10,2),
        change_2000 DECIMAL(10,2)
    );
    """
    cursor.execute(create_table_sql)
    conn.commit()

    for index, row in df.iterrows():
        sql = f"""
        INSERT INTO {table_name} (location, global_total, emission_2023, emission_2000, change_2000)
        VALUES (%s, %s, %s, %s, %s);
        """
        cursor.execute(sql, row.tolist())

    conn.commit()

    engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config.get('port', 5432)}/{db_config['database']}")
    df.to_sql(table_name, engine, if_exists='append', index=False)

except psycopg2.Error as e:
    print(f"Error connecting to or interacting with PostgreSQL: {e}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cursor.close()
        conn.close()
        print("PostgreSQL connection closed.")

An unexpected error occurred: (psycopg2.errors.UndefinedColumn) column "Location" of relation "co2_emission" does not exist
LINE 1: INSERT INTO co2_emission ("Location", "% of global total", "...
                                  ^

[SQL: INSERT INTO co2_emission ("Location", "%% of global total", "Fossil emissions 2023", "Fossil emissions 2000", "%% change from 2000") VALUES (%(Location__0)s, %(P_of_global_total__0)s, %(Fossil_emissions_2023__0)s, %(Fossil_emissions_2000__0)s, %(P_ch ... 29200 characters truncated ... _210)s, %(Fossil_emissions_2023__210)s, %(Fossil_emissions_2000__210)s, %(P_change_from_2000__210)s)]
[parameters: {'P_change_from_2000__0': 262.0, 'Fossil_emissions_2000__0': 3666.95, 'Fossil_emissions_2023__0': 13259.64, 'P_of_global_total__0': 34.0, 'Location__0': 'China', 'P_change_from_2000__1': -21.0, 'Fossil_emissions_2000__1': 5928.97, 'Fossil_emissions_2023__1': 4682.04, 'P_of_global_total__1': 12.0, 'Location__1': 'United States', 'P_change_from_2000__2': 197.0